In [33]:
import math

In [34]:
# Define MSA values (Mean Species Abundance) for all land use types and intensities based on Table 3
# from the True Price biodiversity module, where MSA = 1 - biodiversity loss coefficient (1-MSA)

msa_values_by_land_use_and_intensity = {
    # Natural forest
    "natural_forest_minimal": 1 - 0.15,   
    "natural_forest_light": 1 - 0.3,      
    "natural_forest_intense": 1 - 0.5,     

    # Plantation forest
    "plantation_forest_minimal": 1 - 0.7,
    "plantation_forest_light": 1 - 0.75,
    "plantation_forest_intense": 1 - 0.8,

    # Cropland
    "cropland_minimal": 1 - 0.6,
    "cropland_light": 1 - 0.7,
    "cropland_intense": 1 - 0.9,

    # Pasture
    "pasture_light": 1 - 0.2,
    "pasture_moderate": 1 - 0.4,
    "pasture_heavy": 1 - 0.7,

    # Urban (assumed fixed)
    "urban": 1 - 0.95
}

# Define biome values from the True Price paper (€/MSA.ha.year)
biome_ess_values = {
    'grassland': 2427,
    'temperate_forest': 1014,
    'tropical_forest': 2118,
    'wetland_inland': 14871,
    'wetland_coastal': 10939,
    'shrubland': 1369
}

# Define BIOMEb shares for the Netherlands (from the True Price paper, Table 2, page 8)
biome_shares = {
    'grassland': 0.68,
    'temperate_forest': 0.14,
    'wetland_inland': 0.08,
    'tropical_forest': 0.0,
    'wetland_coastal': 0.4,
    'woodlands': 0.6
}


In [ ]:

def biodiversity_valuation_pipeline(
    land_use_type: str,
    land_size: float,
    unit_product: float,
    species_counts: list,
    age_food_forest: int,
    biome_shares: dict,
    biome_ess_values: dict,
    msa_values_lookup: dict,
    H_agriculture: float = 1.25,    # estimated average shannon
    H_nature: float = 3.0,          # estimated average shannon
    msa_agriculture: float = 0.23,
    msa_nature: float = 1.0
):
    
# Step 1: Shannon Index
    total_species = sum(species_counts)
    if total_species == 0:
        H_ff = 0.0
    else:
        H_ff = -sum((c / total_species) * math.log(c / total_species) for c in species_counts if c > 0)


# Step 2: Relative biodiversity score
    rel_score = (H_ff - H_agriculture) / (H_nature - H_agriculture)
    rel_score = max(0, min(rel_score, 1))  # between 0 and 1


# Step 3: Estimate MSA from relative score
    estimated_msa = msa_agriculture + rel_score * (msa_nature - msa_agriculture)

# Step 4: Calculate €/ha/year
    land_use_per_unit_output = land_size / unit_product

    value_loss = sum(biome_shares[biome] * biome_ess_values.get(biome, 0) * (1 - estimated_msa) for biome in biome_shares)


# Step 5: Convert to €/kg
    euro_per_kg = value_loss * land_use_per_unit_output

    return {
        "Shannon Index": round(H_ff, 3),
        "Relative score": round(rel_score, 3),
        "Estimated MSA": round(estimated_msa, 3),
        "Euro per hectare": round(value_loss, 2),
        "Euro per KG": round(euro_per_kg, 4)
    }


# Example usage:
results = biodiversity_valuation_pipeline(
    land_use_type="cropland_intense",       # 
    land_size = 1,                # ha
    unit_product = 5000,     # kg
    species_counts=[10, 15, 5, 20, 20, 50, 13, 4, 21],
    age_food_forest=10,
    biome_shares=biome_shares,
    biome_ess_values=biome_ess_values,
    msa_values_lookup=msa_values_by_land_use_and_intensity
)

# print results
for key, value in results.items():
    print(f"{key}: {value}")


Shannon Index: 1.962
Relative score: 0.407
Estimated MSA: 0.543
Euro per hectare: 3361.48
Euro per KG: 0.6723
